In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
df = pd.read_csv(r'C:\Users\edoar\Documents\dataset.csv')

In [3]:
df = df.drop(['Unnamed: 0',
 'album_name',
 'track_name'], axis=1)

In [4]:
df=df.dropna().drop_duplicates(subset=['track_id']).set_index('track_id')

In [5]:
df

,artists,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
track_id,,,,,,,,,,,,,,,,,
5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
5vjLSffimiIP26QG5WcN2K,Chord Overstreet,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music


In [6]:
train, test = train_test_split(df, test_size=.2, random_state=1)

In [7]:
predictors = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness",
             "valence", "tempo"]
target = "popularity"

In [8]:
def ridge_fit(train, predictors, target, alpha):
    X = train[predictors].copy()
    y = train[[target]].copy()
    
    y_mean = y.mean()
    y = y - y_mean  # Center the target variable
    
    x_mean = X.mean()
    x_std = X.std()  
    
    X = (X - x_mean) / x_std  # Standardize predictors
    
    penalty = alpha * np.identity(X.shape[1]) 

    # Ridge regression formula
    B = np.linalg.inv(X.T @ X + penalty) @ X.T @ y
    diagonal_matrix = np.diag(1 / x_std)
    beta_hat = diagonal_matrix @ B
    beta_hat.index =  ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness",
             "valence", "tempo"]
    
    return beta_hat, x_mean, x_std, y_mean

def ridge_predict(test, predictors, x_mean, x_std, beta_hat, y_mean):
    # Extract and standardize the test predictors
    test_X = test[predictors]
    test_X_standardized = (test_X - x_mean) / x_std
    
    # Make predictions
    predictions = test_X_standardized @ beta_hat
    
    # Add the mean of the target variable to the predictions to reverse the earlier mean centering
    predictions = predictions + y_mean
    
    return predictions

In [9]:
alphas = [20**i for i in range(-8, 8)]

In [10]:
alphas

[3.90625e-11,
 7.8125e-10,
 1.5625e-08,
 3.125e-07,
 6.25e-06,
 0.000125,
 0.0025,
 0.05,
 1,
 20,
 400,
 8000,
 160000,
 3200000,
 64000000,
 1280000000]

In [11]:
kf = KFold(n_splits=5)
# Initialize dictionaries to store errors for each alpha
errors_dict_mae = {}
errors_dict_mse = {}
errors_dict_rmse = {}
errors_dict_r2 = {}

# Iterate over alpha values
for alpha in alphas:
    fold_errors_mae = []
    fold_errors_mse = []
    fold_errors_rmse = []
    fold_errors_r2 = []
    
    # Perform cross-validation
    for train_index, val_index in kf.split(train):
        train_fold = train.iloc[train_index]
        val_fold = train.iloc[val_index]
        
        beta_hat, x_mean, x_std, y_mean = ridge_fit(train_fold, predictors, target, alpha)
        predictions = ridge_predict(val_fold, predictors, x_mean, x_std, beta_hat, y_mean)
        
        fold_error_mae = mean_absolute_error(val_fold[target], predictions)
        fold_error_mse = mean_squared_error(val_fold[target], predictions)
        fold_error_rmse = np.sqrt(fold_error_mse)  
        fold_error_r2 = r2_score(val_fold[target], predictions)
        
        fold_errors_mae.append(fold_error_mae)
        fold_errors_mse.append(fold_error_mse)
        fold_errors_rmse.append(fold_error_rmse)
        fold_errors_r2.append(fold_error_r2)
    
    # Store the lists of errors for this alpha in the dictionaries
    errors_dict_mae[alpha] = fold_errors_mae
    errors_dict_mse[alpha] = fold_errors_mse
    errors_dict_rmse[alpha] = fold_errors_rmse
    errors_dict_r2[alpha] = fold_errors_r2

# Find the best alpha
best_alpha_mae = min(errors_dict_mae, key=lambda k: np.mean(errors_dict_mae[k]))
best_alpha_mse = min(errors_dict_mse, key=lambda k: np.mean(errors_dict_mse[k]))
best_alpha_rmse = min(errors_dict_rmse, key=lambda k: np.mean(errors_dict_rmse[k]))
best_alpha_r2 = max(errors_dict_r2, key=lambda k: np.mean(errors_dict_r2[k]))  

# Get the corresponding errors for the best alphas
best_alpha_errors_mae = errors_dict_mae[best_alpha_mae]
best_alpha_errors_mse = errors_dict_mse[best_alpha_mse]
best_alpha_errors_rmse = errors_dict_rmse[best_alpha_rmse]
best_alpha_errors_r2 = errors_dict_r2[best_alpha_r2]

print("Best Alpha (MAE):", best_alpha_mae)
print("Errors for Best Alpha (MAE):", best_alpha_errors_mae)
print()
print("Best Alpha (MSE):", best_alpha_mse)
print("Errors for Best Alpha (MSE):", best_alpha_errors_mse)
print()
print("Best Alpha (RMSE):", best_alpha_rmse)
print("Errors for Best Alpha (RMSE):", best_alpha_errors_rmse)
print()
print("Best Alpha (R²):", best_alpha_r2)
print("Errors for Best Alpha (R²):", best_alpha_errors_r2)

Best Alpha (MAE): 160000
Errors for Best Alpha (MAE): [16.984099231809452, 16.7791581991796, 16.76323872614074, 16.842715404578133, 16.98221289216527]

Best Alpha (MSE): 160000
Errors for Best Alpha (MSE): [418.4250644825053, 410.55151921470537, 411.59033767270995, 416.4435110463143, 421.2116677280815]

Best Alpha (RMSE): 160000
Errors for Best Alpha (RMSE): [20.45544095057609, 20.26207095078648, 20.28768931329317, 20.40694761708165, 20.523441907440418]

Best Alpha (R²): 160000
Errors for Best Alpha (R²): [0.018629369079521507, 0.025700320138881017, 0.024269058367557994, 0.021391996519918743, 0.016716101511145998]


In [12]:
# Initialize a dictionary to store average metric values for each alpha
average_metrics_dict = {
    'Alpha': alphas,
    'MAE': [],
    'MSE': [],
    'RMSE': [],
    'R2': []
}

# Iterate over alpha values
for alpha in alphas:
    fold_errors_mae = []
    fold_errors_mse = []
    fold_errors_rmse = []
    fold_errors_r2 = []
    
    # Perform cross-validation
    for train_index, val_index in kf.split(train):
        train_fold = train.iloc[train_index]
        val_fold = train.iloc[val_index]
        
        beta_hat, x_mean, x_std, y_mean = ridge_fit(train_fold, predictors, target, alpha)
        predictions = ridge_predict(val_fold, predictors, x_mean, x_std, beta_hat, y_mean)
        
        # Calculate evaluation metrics
        fold_error_mae = mean_absolute_error(val_fold[target], predictions)
        fold_error_mse = mean_squared_error(val_fold[target], predictions)
        fold_error_rmse = np.sqrt(fold_error_mse) 
        fold_error_r2 = r2_score(val_fold[target], predictions)
        
        fold_errors_mae.append(fold_error_mae)
        fold_errors_mse.append(fold_error_mse)
        fold_errors_rmse.append(fold_error_rmse)
        fold_errors_r2.append(fold_error_r2)
    
    # Calculate and store the average metric values for this alpha
    average_metrics_dict['MAE'].append(np.mean(fold_errors_mae))
    average_metrics_dict['MSE'].append(np.mean(fold_errors_mse))
    average_metrics_dict['RMSE'].append(np.mean(fold_errors_rmse))
    average_metrics_dict['R2'].append(np.mean(fold_errors_r2))

# Create a DataFrame from the dictionary
average_metrics_df = pd.DataFrame(average_metrics_dict)

# Sort the DataFrame by MAE for better analysis
average_metrics_df = average_metrics_df.sort_values(by='MAE')

# Display the DataFrame
print(average_metrics_df)

           Alpha        MAE         MSE       RMSE        R2
12  1.600000e+05  16.870285  415.644420  20.387118  0.021341
13  3.200000e+06  17.209734  423.289230  20.573887  0.003321
14  6.400000e+07  17.249144  424.648073  20.606888  0.000120
15  1.280000e+09  17.251210  424.720827  20.608654 -0.000051
11  8.000000e+03  18.090290  519.022802  22.779860 -0.221940
10  4.000000e+02  18.983122  584.040172  24.163518 -0.374973
9   2.000000e+01  19.049082  588.827294  24.262261 -0.386240
8   1.000000e+00  19.052462  589.072427  24.267306 -0.386817
7   5.000000e-02  19.052632  589.084698  24.267559 -0.386846
6   2.500000e-03  19.052640  589.085312  24.267572 -0.386848
5   1.250000e-04  19.052641  589.085343  24.267572 -0.386848
4   6.250000e-06  19.052641  589.085344  24.267572 -0.386848
3   3.125000e-07  19.052641  589.085344  24.267572 -0.386848
2   1.562500e-08  19.052641  589.085344  24.267572 -0.386848
1   7.812500e-10  19.052641  589.085344  24.267572 -0.386848
0   3.906250e-11  19.052

In [13]:
# Train the ridge regression model on the entire training set with the best alpha
best_alpha = 160000
beta_hat, x_mean, x_std, y_mean = ridge_fit(train, predictors, target, best_alpha)

# Make predictions on the test set
test_predictions = ridge_predict(test, predictors, x_mean, x_std, beta_hat, y_mean)

# Evaluate the model on the test set
mae_test = mean_absolute_error(test[target], test_predictions)
mse_test = mean_squared_error(test[target], test_predictions)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(test[target], test_predictions)

# Print the evaluation metrics
print("Test Set MAE:", mae_test)
print("Test Set MSE:", mse_test)
print("Test Set RMSE:", rmse_test)
print("Test Set R²:", r2_test)

Test Set MAE: 16.76948934448852
Test Set MSE: 410.8299718951608
Test Set RMSE: 20.268941064968363
Test Set R²: 0.019305070169372107


In [14]:
beta_hat.sort_values(by='popularity', ascending=False)

,popularity
danceability,2.147739
loudness,0.063019
key,0.004328
tempo,0.000233
energy,-0.211382
mode,-0.269285
liveness,-0.395533
acousticness,-0.490696
valence,-0.886250
instrumentalness,-2.406238


The model indicates that songs with greater danceability are more likely to gain popularity. Conversely, songs with excessive speechiness or instrumental elements are less likely to achieve the same level of popularity.

## Categorical variables

In [15]:
encoded_df = pd.get_dummies(df, columns=['track_genre', "explicit"])

In [16]:
encoded_df

,artists,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,...,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_world-music,explicit_False,explicit_True
track_id,,,,,,,,,,,,,,,,,,,,,
5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,73,230666,0.676,0.4610,1,-6.746,0,0.1430,0.0322,...,0,0,0,0,0,0,0,0,1,0
4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,55,149610,0.420,0.1660,1,-17.235,1,0.0763,0.9240,...,0,0,0,0,0,0,0,0,1,0
1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,57,210826,0.438,0.3590,0,-9.734,1,0.0557,0.2100,...,0,0,0,0,0,0,0,0,1,0
6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,71,201933,0.266,0.0596,0,-18.515,1,0.0363,0.9050,...,0,0,0,0,0,0,0,0,1,0
5vjLSffimiIP26QG5WcN2K,Chord Overstreet,82,198853,0.618,0.4430,2,-9.681,1,0.0526,0.4690,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,21,384999,0.172,0.2350,5,-16.393,1,0.0422,0.6400,...,0,0,0,0,0,0,0,1,1,0
1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,22,385000,0.174,0.1170,0,-18.318,0,0.0401,0.9940,...,0,0,0,0,0,0,0,1,1,0
6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,22,271466,0.629,0.3290,0,-10.895,0,0.0420,0.8670,...,0,0,0,0,0,0,0,1,1,0


In [17]:
new_df = encoded_df.drop(['artists'], axis=1)

In [18]:
train, test = train_test_split(new_df, test_size=.2, random_state=1)

In [19]:
predictors = ['track_genre_acoustic',
 'track_genre_afrobeat',
 'track_genre_alt-rock',
 'track_genre_alternative',
 'track_genre_ambient',
 'track_genre_anime',
 'track_genre_black-metal',
 'track_genre_bluegrass',
 'track_genre_blues',
 'track_genre_brazil',
 'track_genre_breakbeat',
 'track_genre_british',
 'track_genre_cantopop',
 'track_genre_chicago-house',
 'track_genre_children',
 'track_genre_chill',
 'track_genre_classical',
 'track_genre_club',
 'track_genre_comedy',
 'track_genre_country',
 'track_genre_dance',
 'track_genre_dancehall',
 'track_genre_death-metal',
 'track_genre_deep-house',
 'track_genre_detroit-techno',
 'track_genre_disco',
 'track_genre_disney',
 'track_genre_drum-and-bass',
 'track_genre_dub',
 'track_genre_dubstep',
 'track_genre_edm',
 'track_genre_electro',
 'track_genre_electronic',
 'track_genre_emo',
 'track_genre_folk',
 'track_genre_forro',
 'track_genre_french',
 'track_genre_funk',
 'track_genre_garage',
 'track_genre_german',
 'track_genre_gospel',
 'track_genre_goth',
 'track_genre_grindcore',
 'track_genre_groove',
 'track_genre_grunge',
 'track_genre_guitar',
 'track_genre_happy',
 'track_genre_hard-rock',
 'track_genre_hardcore',
 'track_genre_hardstyle',
 'track_genre_heavy-metal',
 'track_genre_hip-hop',
 'track_genre_honky-tonk',
 'track_genre_house',
 'track_genre_idm',
 'track_genre_indian',
 'track_genre_indie',
 'track_genre_indie-pop',
 'track_genre_industrial',
 'track_genre_iranian',
 'track_genre_j-dance',
 'track_genre_j-idol',
 'track_genre_j-pop',
 'track_genre_j-rock',
 'track_genre_jazz',
 'track_genre_k-pop',
 'track_genre_kids',
 'track_genre_latin',
 'track_genre_latino',
 'track_genre_malay',
 'track_genre_mandopop',
 'track_genre_metal',
 'track_genre_metalcore',
 'track_genre_minimal-techno',
 'track_genre_mpb',
 'track_genre_new-age',
 'track_genre_opera',
 'track_genre_pagode',
 'track_genre_party',
 'track_genre_piano',
 'track_genre_pop',
 'track_genre_pop-film',
 'track_genre_power-pop',
 'track_genre_progressive-house',
 'track_genre_psych-rock',
 'track_genre_punk',
 'track_genre_punk-rock',
 'track_genre_r-n-b',
 'track_genre_reggae',
 'track_genre_reggaeton',
 'track_genre_rock',
 'track_genre_rock-n-roll',
 'track_genre_rockabilly',
 'track_genre_romance',
 'track_genre_sad',
 'track_genre_salsa',
 'track_genre_samba',
 'track_genre_sertanejo',
 'track_genre_show-tunes',
 'track_genre_singer-songwriter',
 'track_genre_ska',
 'track_genre_sleep',
 'track_genre_soul',
 'track_genre_spanish',
 'track_genre_study',
 'track_genre_swedish',
 'track_genre_synth-pop',
 'track_genre_tango',
 'track_genre_techno',
 'track_genre_trance',
 'track_genre_trip-hop',
 'track_genre_turkish',
 'track_genre_world-music',
 'explicit_False',
 'explicit_True',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo']
target = "popularity"

In [20]:
def ridge_fit(train, predictors, target, alpha):
    X = train[predictors].copy()
    y = train[[target]].copy()
    
    y_mean = y.mean()
    y = y - y_mean  # Center the target variable
    
    x_mean = X.mean()
    x_std = X.std() 
    
    X = (X - x_mean) / x_std  # Standardize predictors
    
    penalty = alpha * np.identity(X.shape[1]) 

    # Ridge regression formula
    B = np.linalg.inv(X.T @ X + penalty) @ X.T @ y
    diagonal_matrix = np.diag(1 / x_std)
    beta_hat = diagonal_matrix @ B
    beta_hat.index =  ['track_genre_acoustic',
 'track_genre_afrobeat',
 'track_genre_alt-rock',
 'track_genre_alternative',
 'track_genre_ambient',
 'track_genre_anime',
 'track_genre_black-metal',
 'track_genre_bluegrass',
 'track_genre_blues',
 'track_genre_brazil',
 'track_genre_breakbeat',
 'track_genre_british',
 'track_genre_cantopop',
 'track_genre_chicago-house',
 'track_genre_children',
 'track_genre_chill',
 'track_genre_classical',
 'track_genre_club',
 'track_genre_comedy',
 'track_genre_country',
 'track_genre_dance',
 'track_genre_dancehall',
 'track_genre_death-metal',
 'track_genre_deep-house',
 'track_genre_detroit-techno',
 'track_genre_disco',
 'track_genre_disney',
 'track_genre_drum-and-bass',
 'track_genre_dub',
 'track_genre_dubstep',
 'track_genre_edm',
 'track_genre_electro',
 'track_genre_electronic',
 'track_genre_emo',
 'track_genre_folk',
 'track_genre_forro',
 'track_genre_french',
 'track_genre_funk',
 'track_genre_garage',
 'track_genre_german',
 'track_genre_gospel',
 'track_genre_goth',
 'track_genre_grindcore',
 'track_genre_groove',
 'track_genre_grunge',
 'track_genre_guitar',
 'track_genre_happy',
 'track_genre_hard-rock',
 'track_genre_hardcore',
 'track_genre_hardstyle',
 'track_genre_heavy-metal',
 'track_genre_hip-hop',
 'track_genre_honky-tonk',
 'track_genre_house',
 'track_genre_idm',
 'track_genre_indian',
 'track_genre_indie',
 'track_genre_indie-pop',
 'track_genre_industrial',
 'track_genre_iranian',
 'track_genre_j-dance',
 'track_genre_j-idol',
 'track_genre_j-pop',
 'track_genre_j-rock',
 'track_genre_jazz',
 'track_genre_k-pop',
 'track_genre_kids',
 'track_genre_latin',
 'track_genre_latino',
 'track_genre_malay',
 'track_genre_mandopop',
 'track_genre_metal',
 'track_genre_metalcore',
 'track_genre_minimal-techno',
 'track_genre_mpb',
 'track_genre_new-age',
 'track_genre_opera',
 'track_genre_pagode',
 'track_genre_party',
 'track_genre_piano',
 'track_genre_pop',
 'track_genre_pop-film',
 'track_genre_power-pop',
 'track_genre_progressive-house',
 'track_genre_psych-rock',
 'track_genre_punk',
 'track_genre_punk-rock',
 'track_genre_r-n-b',
 'track_genre_reggae',
 'track_genre_reggaeton',
 'track_genre_rock',
 'track_genre_rock-n-roll',
 'track_genre_rockabilly',
 'track_genre_romance',
 'track_genre_sad',
 'track_genre_salsa',
 'track_genre_samba',
 'track_genre_sertanejo',
 'track_genre_show-tunes',
 'track_genre_singer-songwriter',
 'track_genre_ska',
 'track_genre_sleep',
 'track_genre_soul',
 'track_genre_spanish',
 'track_genre_study',
 'track_genre_swedish',
 'track_genre_synth-pop',
 'track_genre_tango',
 'track_genre_techno',
 'track_genre_trance',
 'track_genre_trip-hop',
 'track_genre_turkish',
 'track_genre_world-music',
 'explicit_False',
 'explicit_True',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo']
    
    return beta_hat, x_mean, x_std, y_mean

In [21]:
def ridge_predict(test, predictors, x_mean, x_std, beta_hat, y_mean):
    # Extract and standardize the test predictors
    test_X = test[predictors]
    test_X_standardized = (test_X - x_mean) / x_std
    
    # Make predictions
    predictions = test_X_standardized @ beta_hat
    
    # Add the mean of the target variable to the predictions to reverse the earlier mean centering
    predictions = predictions + y_mean
    
    return predictions

In [22]:
alphas = [20**i for i in range(-8, 8)]

In [23]:
kf = KFold(n_splits=5)
# Initialize dictionaries to store errors for each alpha
errors_dict_mae = {}
errors_dict_mse = {}
errors_dict_rmse = {}
errors_dict_r2 = {}

# Iterate over alpha values
for alpha in alphas:
    fold_errors_mae = []
    fold_errors_mse = []
    fold_errors_rmse = []
    fold_errors_r2 = []
    
    # Perform cross-validation
    for train_index, val_index in kf.split(train):
        train_fold = train.iloc[train_index]
        val_fold = train.iloc[val_index]
        
        beta_hat, x_mean, x_std, y_mean = ridge_fit(train_fold, predictors, target, alpha)
        predictions = ridge_predict(val_fold, predictors, x_mean, x_std, beta_hat, y_mean)
        
        # Calculate evaluation metrics
        fold_error_mae = mean_absolute_error(val_fold[target], predictions)
        fold_error_mse = mean_squared_error(val_fold[target], predictions)
        fold_error_rmse = np.sqrt(fold_error_mse)  
        fold_error_r2 = r2_score(val_fold[target], predictions)
        
        fold_errors_mae.append(fold_error_mae)
        fold_errors_mse.append(fold_error_mse)
        fold_errors_rmse.append(fold_error_rmse)
        fold_errors_r2.append(fold_error_r2)
    
    # Store the lists of errors for this alpha in the dictionaries
    errors_dict_mae[alpha] = fold_errors_mae
    errors_dict_mse[alpha] = fold_errors_mse
    errors_dict_rmse[alpha] = fold_errors_rmse
    errors_dict_r2[alpha] = fold_errors_r2

# Find the best alpha
best_alpha_mae = min(errors_dict_mae, key=lambda k: np.mean(errors_dict_mae[k]))
best_alpha_mse = min(errors_dict_mse, key=lambda k: np.mean(errors_dict_mse[k]))
best_alpha_rmse = min(errors_dict_rmse, key=lambda k: np.mean(errors_dict_rmse[k]))
best_alpha_r2 = max(errors_dict_r2, key=lambda k: np.mean(errors_dict_r2[k]))

# Get the corresponding errors for the best alphas
best_alpha_errors_mae = errors_dict_mae[best_alpha_mae]
best_alpha_errors_mse = errors_dict_mse[best_alpha_mse]
best_alpha_errors_rmse = errors_dict_rmse[best_alpha_rmse]
best_alpha_errors_r2 = errors_dict_r2[best_alpha_r2]

print("Best Alpha (MAE):", best_alpha_mae)
print("Errors for Best Alpha (MAE):", best_alpha_errors_mae)
print()
print("Best Alpha (MSE):", best_alpha_mse)
print("Errors for Best Alpha (MSE):", best_alpha_errors_mse)
print()
print("Best Alpha (RMSE):", best_alpha_rmse)
print("Errors for Best Alpha (RMSE):", best_alpha_errors_rmse)
print()
print("Best Alpha (R²):", best_alpha_r2)
print("Errors for Best Alpha (R²):", best_alpha_errors_r2)

Best Alpha (MAE): 3200000
Errors for Best Alpha (MAE): [16.120730292852166, 15.965346149315435, 15.937759938128117, 16.00146075813996, 16.088692935812475]

Best Alpha (MSE): 3200000
Errors for Best Alpha (MSE): [379.19953500361936, 373.710346941826, 373.7848752777195, 378.3173697618031, 380.9352683909167]

Best Alpha (RMSE): 3200000
Errors for Best Alpha (RMSE): [19.473046371937272, 19.331589353744974, 19.33351688849495, 19.450382252331266, 19.517563075110495]

Best Alpha (R²): 3200000
Errors for Best Alpha (R²): [0.11062859637364464, 0.1131298890754, 0.11389205493762489, 0.11098529312116079, 0.11073803393042458]


In [24]:
# Initialize a dictionary to store average metric values for each alpha
average_metrics_dict = {
    'Alpha': alphas,
    'MAE': [],
    'MSE': [],
    'RMSE': [],
    'R2': []
}

# Iterate over alpha values
for alpha in alphas:
    fold_errors_mae = []
    fold_errors_mse = []
    fold_errors_rmse = []
    fold_errors_r2 = []
    
    # Perform cross-validation
    for train_index, val_index in kf.split(train):
        train_fold = train.iloc[train_index]
        val_fold = train.iloc[val_index]
        
        beta_hat, x_mean, x_std, y_mean = ridge_fit(train_fold, predictors, target, alpha)
        predictions = ridge_predict(val_fold, predictors, x_mean, x_std, beta_hat, y_mean)
        
        # Calculate evaluation metrics
        fold_error_mae = mean_absolute_error(val_fold[target], predictions)
        fold_error_mse = mean_squared_error(val_fold[target], predictions)
        fold_error_rmse = np.sqrt(fold_error_mse)  # RMSE is the square root of MSE
        fold_error_r2 = r2_score(val_fold[target], predictions)
        
        fold_errors_mae.append(fold_error_mae)
        fold_errors_mse.append(fold_error_mse)
        fold_errors_rmse.append(fold_error_rmse)
        fold_errors_r2.append(fold_error_r2)
    
    # Calculate and store the average metric values for this alpha
    average_metrics_dict['MAE'].append(np.mean(fold_errors_mae))
    average_metrics_dict['MSE'].append(np.mean(fold_errors_mse))
    average_metrics_dict['RMSE'].append(np.mean(fold_errors_rmse))
    average_metrics_dict['R2'].append(np.mean(fold_errors_r2))

# Create a DataFrame from the dictionary
average_metrics_df = pd.DataFrame(average_metrics_dict)

# Sort the DataFrame by MAE 
average_metrics_df = average_metrics_df.sort_values(by='MAE')

# Display the DataFrame
print(average_metrics_df)


           Alpha        MAE           MSE        RMSE         R2
13  3.200000e+06  16.022798    377.189479   19.421220   0.111875
14  6.400000e+07  17.186918    422.047961   20.543702   0.006243
15  1.280000e+09  17.248095    424.590055   20.605481   0.000257
12  1.600000e+05  21.415109    768.653764   27.722202  -0.809997
11  8.000000e+03  78.110259   9648.307002   98.218649 -21.720162
0   3.906250e-11  88.380888  12719.356390  112.769229 -28.945372
10  4.000000e+02  89.258610  12609.878357  112.285681 -28.694150
9   2.000000e+01  89.903475  12793.072884  113.098383 -29.125539
8   1.000000e+00  89.935981  12802.340445  113.139341 -29.147362
7   5.000000e-02  89.937607  12802.804096  113.141390 -29.148454
6   2.500000e-03  89.937688  12802.827281  113.141492 -29.148509
4   6.250000e-06  89.937692  12802.828538  113.141498 -29.148512
5   1.250000e-04  89.937692  12802.828472  113.141497 -29.148511
3   3.125000e-07  89.937710  12802.834374  113.141527 -29.148525
2   1.562500e-08  89.9383

In [25]:
# Train the ridge regression model on the entire training set with the best alpha
best_alpha = 3200000
beta_hat, x_mean, x_std, y_mean = ridge_fit(train, predictors, target, best_alpha)

# Make predictions on the test set
test_predictions = ridge_predict(test, predictors, x_mean, x_std, beta_hat, y_mean)

# Evaluate the model on the test set
mae_test = mean_absolute_error(test[target], test_predictions)
mse_test = mean_squared_error(test[target], test_predictions)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(test[target], test_predictions)

# Print the evaluation metrics
print("Test Set MAE:", mae_test)
print("Test Set MSE:", mse_test)
print("Test Set RMSE:", rmse_test)
print("Test Set R²:", r2_test)

Test Set MAE: 15.631488830613298
Test Set MSE: 360.307791248582
Test Set RMSE: 18.981775239649796
Test Set R²: 0.13990690010774542


In [26]:
beta_hat.sort_values(by='popularity', ascending=False).head(10)

,popularity
track_genre_k-pop,0.574352
track_genre_pop-film,0.572595
track_genre_metal,0.505547
track_genre_chill,0.451657
track_genre_latino,0.397910
track_genre_sad,0.393906
track_genre_grunge,0.387021
track_genre_indian,0.368940
track_genre_emo,0.350888
track_genre_anime,0.345318


In [27]:
beta_hat.sort_values(by='popularity', ascending=False).tail(10)

,popularity
track_genre_idm,-0.394483
track_genre_kids,-0.403672
track_genre_grindcore,-0.414993
track_genre_classical,-0.435329
track_genre_chicago-house,-0.458817
track_genre_detroit-techno,-0.493323
track_genre_latin,-0.507894
track_genre_jazz,-0.548960
track_genre_romance,-0.654976
track_genre_iranian,-0.682987


The model indicates that songs belonging to the genres of K-pop, pop-film, metal, chill or Latino are more likely to gain popularity. Conversely, songs belonging to genres such as IDM, kids music, grindcore, classical music, or Chicago house are less likely to achieve the same level of popularity.

**Numerical Model Training Metrics**:
- MAE: 16.87 
- MSE: 415.64 
- RMSE: 20.39 
- R²: 0.021 

**Numerical Model Test Metrics**:
- MAE: 16.77 
- MSE: 410.83 
- RMSE: 20.27 
- R²: 0.019

The training and test errors (MAE, MSE, RMSE) are very similar, indicating that the model's performance is consistent between the training and test datasets. 
The R² scores are extremely low for both training (0.021) and test (0.019) datasets, suggesting that the model does not capture much variance in the data. 

The very low R² scores and similar error metrics suggest underfitting. The model is too simple and fails to capture the underlying patterns in the data. 

**Complete Model Training Metrics**:
- MAE: 16.02 
- MSE: 377.19
- RMSE: 19.42
- R²: 0.112 

**Complete Model Test Metrics**:
- MAE: 15.63 
- MSE: 360.31
- RMSE: 18.98
- R²: 0.140

The training errors are slightly higher than the test errors, but the differences are not substantial, which indicates that the model performs consistently on both datasets. 
The R² score is moderately low for training (0.112) and test (0.140) datasets, but it is higher than that of the numerical model. This indicates that the model explains a bit more variance in the data but still lacks a strong fit. 

The slightly better performance on the test set could suggest that the model generalizes fairly well, but the relatively low R² scores still indicate some degree of underfitting. The model captures more patterns than the numerical model but still fails to explain a substantial portion of the variance. 

This model shows slight signs of underfitting as well, but it performs better. The relatively low but higher R² scores suggest that it is capturing more variance but still not enough to be considered a good fit.

With regard to the encoded variable ‘avg_artist_pop’, the rationale behind it is that there is a higher probability of a song being popular if the artist releasing it is popular. It is evident that the metrics alone of a song are not sufficient to explain popularity.\
The issue with this dataset and the consequent creation of data leakage is the small number of multiple songs for individual artists. This is something that is applicable in the real world but not in a dataset containing 87.000 songs.\
Given the limited number of rows in the dataset and the data leakage issue, I have decided to remove it from the project, despite the fact that I consider it to be a valid proposal for a different dataset.